In [1]:
import numpy as np

In [2]:
from scipy import *

In [3]:
import scipy.integrate as scintegrate

In [4]:
import scipy.special as scispecial

In [5]:
from scipy.misc import derivative

In [6]:
from sympy import *

In [7]:
from time import time

In [8]:
def myF(x):
    return abs(np.sin(x))

In [9]:
RealI = scintegrate.quad(myF, -2, 2)[0]
RealI

2.8322936730942847

In [10]:
def midRect(a, b, n):
    tic = time()
    X = np.linspace(a, b, n)[:-1]
    Y = 0
    for i in X: Y += myF(i+((b-a)/(2*n)))
    I = ((b-a)/n)*(Y)
    toc = time()
    #print("RunTime: ", toc-tic)
    #print("Difference: ", abs(RealI - I))
    return I

In [11]:
midRect(-2, 2, 3000)

2.8313493407764199

In [11]:
def Trapz(a, b, n):
    tic = time()
    X = np.linspace(a, b, n)[1:-1]
    Y = 0
    for i in X: Y += myF(i)
    I = ((b-a)/(2*n))*(myF(a)+2*(Y)+myF(b))
    toc = time()
    print("RunTime: ", toc - tic)
    print("Difference: ", abs(RealI - I))
    return I

In [13]:
Trapz(-2, 2, 3000)

RunTime:  0.011000633239746094
Difference:  0.000944073037354


2.8313496000569307

In [12]:
def Simps(a, b, n):
    tic = time()
    X = np.linspace(a, b, 2*n)
    evY = 0
    unevY = 0
    for i in X[2:-1:2]: evY += myF(i)
    for i in X[1:-1:2]: unevY += myF(i)
    I = ((b-a)/(6*n))*(myF(a) + 4*unevY + 2*evY + myF(b))
    toc = time()
    #print("RunTime: ", toc - tic)
    #print("Difference: ", abs(RealI - I))
    return I

In [15]:
Simps(-2, 2, 3000)

RunTime:  0.02100086212158203
Difference:  0.000674108827982


2.8316195642663029

In [13]:
def Gauss2(a, b):
    #tic = time()
    c1, c2, x1, x2 = symbols("c1, c2, x1, x2")
    Solves = double(solve([c1 + c2 - (b-a), 
                           c1*x1 + c2*x2 - ((b**2 - a**2)/2), 
                           c1*x1**2 + c2*x2**2 - ((b**3 - a**3)/3), 
                           c1*x1**3 + c2*x2**3 - ((b**4 - a**4)/4)], 
                          [c1, c2, x1, x2])[0])
    I = Solves[0]*myF(Solves[2]) + Solves[1]*myF(Solves[3])
    #toc = time()
    #print("RunTime: ", toc - tic)
    return I

In [17]:
Gauss2(-2, 2)

3.6586958412563666

In [14]:
def Gauss3(a, b):
    #tic = time()
    c1, c2, c3, x1, x2, x3 = symbols("c1, c2, c3, x1, x2, x3")
    Solves = double(solve([c1 + c2 + c3 - (b-a), 
                           c1*x1 + c2*x2 + c3*x3- ((b**2 - a**2)/2), 
                           c1*x1**2 + c2*x2**2 + c3*x3**2 - ((b**3 - a**3)/3), 
                           c1*x1**3 + c2*x2**3 + c3*x3**3 - ((b**4 - a**4)/4), 
                           c1*x1**4 + c2*x2**4 + c3*x3**4 - ((b**5 - a**5)/5), 
                           c1*x1**5 + c2*x2**5 + c3*x3**5 - ((b**6 - a**6)/6)], 
                          [c1, c2, c3, x1, x2, x3])[0])
    I = Solves[0]*myF(Solves[3]) + Solves[1]*myF(Solves[4]) + Solves[2]*myF(Solves[5])
    #toc = time()
    #print("RunTime: ", toc - tic)
    return I

In [19]:
Gauss3(-2, 2)

2.221703698939562

In [15]:
def FGauss2(a, b, n):
    tic = time()
    X = np.linspace(a, b, n)
    I = 0
    for i in range(len(X)-1): I += Gauss2(X[i], X[i+1])
    toc = time()
    print("RunTime: ", toc - tic)
    print("Difference: ", abs(RealI - I))
    return I

In [21]:
FGauss2(-2, 2, 3)

RunTime:  0.29701709747314453
Difference:  0.0119797097635


2.8203139633307615

In [16]:
def FGauss3(a, b, n):
    tic = time()
    X = np.linspace(a, b, n)
    I = 0
    for i in range(len(X)-1): I += Gauss3(X[i], X[i+1])
    toc = time()
    #print("RunTime: ", toc - tic)
    #print("Difference: ", abs(RealI - I))
    #print("TheorDifference: ", )
    return I

In [23]:
FGauss3(-2, 2, 3)

RunTime:  1.3670785427093506
Difference:  0.00010363238762
TheorDifference: 


2.8323973054819045

In [17]:
def precision(quadF, n):
    if quadF == FGauss3:
        return 2*n - 1
    elif quadF == FGauss2:
        return 2*n - 1
    elif quadF == Simps:
        return 4
    elif quadF == Trapz:
        return 2
    elif quadF == midRect:
        return 2
    else:
        return 1

In [24]:
precision(FGauss3, 2)

3

In [19]:
def ruleRunge(quadF, a, b):
    n = 1
    epsilon = 1
    iteration = 0
    while (epsilon > 1e-4):
        iteration += 1
        n *= 2
        m = precision(quadF, n)
        epsilon = abs(quadF(a, b, n*2) - quadF(a, b, n))/(2**(m - 1) - 1)
        print("For n =", n, ":", epsilon)
    tic = time()
    I = quadF(a, b, n)
    toc = time()
    print("Convergence:", iteration)
    print()
    print("RunTime:", toc - tic)

In [20]:
ruleRunge(midRect, -2, 2)

For n = 2 : 0.399894128935
For n = 4 : 0.373841224062
For n = 8 : 0.191781843235
For n = 16 : 0.0934424644207
For n = 32 : 0.0456443240108
For n = 64 : 0.0224943370197
For n = 128 : 0.011157861969
For n = 256 : 0.00555568421913
For n = 512 : 0.00277191532694
For n = 1024 : 0.00138446157702
For n = 2048 : 0.000691854967775
For n = 4096 : 0.000345833303797
For n = 8192 : 0.000172893078766
For n = 16384 : 8.6440642606e-05
Convergence: 14

RunTime: 0.03500223159790039
